In [1]:
from __future__ import division
from sys import exit
from math import sqrt
from numpy import array
from scipy.optimize import fmin_l_bfgs_b
from scipy.optimize import basinhopping
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
import os

def MAPE(params, *kwargs):
	Y = kwargs[0]
	type = kwargs[1]
	mape = 0
	if type == 'linear':
		alpha, beta = params
		a = [Y[0]]
		b = [Y[1] - Y[0]]
		y = [a[0] + b[0]]

		for i in range(len(Y)):
			a.append(alpha * Y[i] + (1 - alpha) * (a[i] + b[i]))
			b.append(beta * (a[i + 1] - a[i]) + (1 - beta) * b[i])
			y.append(a[i + 1] + b[i + 1])
	else:
		alpha, beta, gamma = params
		m = kwargs[2]
		a = [sum(Y[0:m]) / float(m)]
		b = [(sum(Y[m:2 * m]) - sum(Y[0:m])) / m ** 2]
        
		if type == 'additive':
			s = [Y[i] - a[0] for i in range(m)]
			y = [a[0] + b[0] + s[0]]
			for i in range(len(Y)):
				a.append(alpha * (Y[i] - s[i]) + (1 - alpha) * (a[i] + b[i]))
				b.append(beta * (a[i + 1] - a[i]) + (1 - beta) * b[i])
				s.append(gamma * (Y[i] - a[i] - b[i]) + (1 - gamma) * s[i])
				y.append(a[i + 1] + b[i + 1] + s[i + 1])

		elif type == 'multiplicative':
			s = [Y[i] / a[0] for i in range(m)]
			y = [(a[0] + b[0]) * s[0]]
			for i in range(len(Y)):
				a.append(alpha * (Y[i] / s[i]) + (1 - alpha) * (a[i] + b[i]))
				b.append(beta * (a[i + 1] - a[i]) + (1 - beta) * b[i])
				s.append(gamma * (Y[i] / (a[i] + b[i])) + (1 - gamma) * s[i])
				y.append((a[i + 1] + b[i + 1]) * s[i + 1])
		else:
			exit('Type must be either linear, additive or multiplicative')

	#rmse = sqrt(sum([(m - n) ** 2 for m, n in zip(Y, y[:-1])]) / len(Y))
	mape = sum([abs((m-n)/float(m)) for m, n in zip(Y, y[:-1])])*100/float(len(Y))
	return mape

In [2]:
def linear(x, fc, alpha = None, beta = None):
	Y = list(x[:])
	if (alpha == None or beta == None):
		initial_values = array([0.3, 0.1])
		boundaries = [(0, 1), (0, 1)]
		type = 'linear'
		xmin = [0.0, 0.0]
		xmax = [1.0, 1.0]
		bounds = [(low, high) for low, high in zip(xmin, xmax)]
		minimizer_kwargs = {"method":"L-BFGS-B", "args" : (Y, type), "bounds" : bounds}
		parameters = basinhopping(MAPE, x0 = initial_values, minimizer_kwargs = minimizer_kwargs,niter = 50)
		alpha, beta = parameters.x[0], parameters.x[1]
        
	a = [Y[0]]
	b = [Y[1] - Y[0]]
	y = [a[0] + b[0]]
	mape = 0
	for i in range(len(Y) + fc):
		if i == len(Y):            
			Y.append(a[-1] + b[-1])
		a.append(alpha * Y[i] + (1 - alpha) * (a[i] + b[i]))
		b.append(beta * (a[i + 1] - a[i]) + (1 - beta) * b[i])
		y.append(a[i + 1] + b[i + 1])

	mape = sum([abs((m-n)/float(m)) for m, n in zip(Y, y[:-1])])*100/float(len(Y[:-fc]))
	return Y,alpha, beta, mape

def additive(x, m, fc, alpha = None, beta = None, gamma = None):
	Y = list(x[:])
	if (alpha == None or beta == None or gamma == None):
		initial_values = array([0.3, 0.1, 0.1])
		boundaries = [(0, 1), (0, 1), (0, 1)]
		type = 'additive'    
		xmin = [0.0, 0.0, 0.0]
		xmax = [1.0, 1.0, 1.0]
		bounds = [(low, high) for low, high in zip(xmin, xmax)]
		minimizer_kwargs = {"method":"L-BFGS-B", "args" : (Y, type, m), "bounds" : bounds}
		parameters = basinhopping(MAPE, x0 = initial_values, minimizer_kwargs = minimizer_kwargs,niter = 50)
		alpha, beta, gamma = parameters.x[0], parameters.x[1], parameters.x[2]
        
	a = [sum(Y[0:m]) / float(m)]
	b = [(sum(Y[m:2 * m]) - sum(Y[0:m])) / m ** 2]
	s = [Y[i] - a[0] for i in range(m)]
	y = [a[0] + b[0] + s[0]]
	mape = 0

	for i in range(len(Y) + fc):
		if i == len(Y):
			Y.append(a[len(a)-1] + b[len(b)-1] + s[len(s)-m])
		a.append(alpha * (Y[i] - s[i]) + (1 - alpha) * (a[i] + b[i]))
		b.append(beta * (a[i + 1] - a[i]) + (1 - beta) * b[i])
		s.append(gamma * (Y[i] - a[i] - b[i]) + (1 - gamma) * s[i])
		y.append(a[i + 1] + b[i + 1] + s[i + 1])
        
	mape = sum([abs((m-n)/float(m)) for m, n in zip(Y, y[:-1])])*100/float(len(Y[:-fc]))
	return Y,alpha, beta, gamma, mape

def multiplicative(x, m, fc, alpha = None, beta = None, gamma = None):
	Y = list(x[:])
	if (alpha == None or beta == None or gamma == None):
		initial_values = array([0.2, 0.05, 0.15])
		type = 'multiplicative'
		xmin = [0.0, 0.0, 0.0]
		xmax = [1.0, 1.0, 1.0]
		bounds = [(low, high) for low, high in zip(xmin, xmax)]
		minimizer_kwargs = {"method":"L-BFGS-B", "args" : (Y, type, m), "bounds" : bounds}
		parameters = basinhopping(MAPE, x0 = initial_values, minimizer_kwargs = minimizer_kwargs,niter = 50)
		alpha, beta, gamma = parameters.x[0], parameters.x[1], parameters.x[2]

	a = [sum(Y[0:m]) / float(m)]
	b = [(sum(Y[m:2 * m]) - sum(Y[0:m])) / m ** 2]
	s = [Y[i] / a[0] for i in range(m)]
	y = [(a[0] + b[0]) * s[0]]
	mape = 0

	for i in range(len(Y) + fc):
		if i == len(Y):
			Y.append((a[len(a)-1] + b[len(b)-1]) * s[len(s)-m])
		a.append(alpha * (Y[i] / s[i]) + (1 - alpha) * (a[i] + b[i]))
		b.append(beta * (a[i + 1] - a[i]) + (1 - beta) * b[i])
		s.append(gamma * (Y[i] / (a[i] + b[i])) + (1 - gamma) * s[i])
		y.append((a[i + 1] + b[i + 1]) * s[i + 1])

	mape = sum([abs((m-n)/float(m)) for m, n in zip(Y, y[:-1])])*100/float(len(Y[:-fc]))
	return Y,alpha, beta, gamma, mape, y[:-1]


In [3]:
def dummy(all_values):
    best_output = []
    min_error = 100.1
    for l in all_values:
        if(l[-1] < min_error):
            min_error = l[-1]
            best_output = l
    return best_output

def write_to_folder(data, opDirectory, filename, actual_dir):
    dataframe = pd.DataFrame()
    dataframe['forecast'] = pd.Series(data)
    os.chdir(opDirectory)
    dataframe.to_csv(filename, sep='\t')
    os.chdir(actual_dir)
    
def check_arguments(l, n):
    arguments , c = None, None
    if(len(l)==4):
        if(l[-1] != -1):
            c = l[-1]
    
        flag = False
        for i in range(n-1):
            if(l[i]== -1):
                flag = True
                break
        if(flag == False):
            arguments = l[:-1]
    elif(len(l)==2):
        flag = False
        for i in range(n):
            if(l[i]== -1):
                flag = True
                break
        if(flag == False):
            arguments = l
    return c, arguments

def prana_forecast( method_dict ):
    os.chdir('D:\\Honors\\Forecasting Algorithms')
    inputcsvfolderpath = "D:\\Honors\\Forecasting Algorithms"
    opDirectory = "D:\\Honors\\Forecasting Algorithms\\NEW_FOLDER"
    inputcsvpath = os.path.join(inputcsvfolderpath, method_dict['filename'])
    
    fn = method_dict['filename'].split('.')
    print (fn)
    actual_dir = os.getcwd()
    os.chdir(opDirectory)
    newDirectory = os.path.join(opDirectory, fn[0])
    os.mkdir(newDirectory)
    os.chdir(actual_dir)
    
    del(method_dict['filename'])
    all_paths = list()
    if(len(method_dict.keys())==0):
        best_mape_path = main_prana(inputcsvpath, newDirectory, None, None, None)
        all_paths.append(best_mape_path)
    else:
        for k in method_dict:
            c, arguments = check_arguments(method_dict[k], len(method_dict[k]))
            csv_path = main_prana(inputcsvpath, newDirectory, c, k, arguments)
            all_paths.append(csv_path)
    return all_paths

def main_prana(inputcsvpath,  opDirectory, c = None, method = None, arguments = None):
    fc = 5
    actual_dir = os.getcwd()
    with open(inputcsvpath, 'r') as f:
        df = pd.read_csv(f)
        df.columns = ['time', 'demand']
        y = df['demand']
        all_values = []
        if(method == None):
            best_fit = {}
            for i in range(int(len(y)//2)):
                output = additive(y, int(i+1), fc)
                temp = [i+1]
                for j in range(0,len(output)):
                    temp.append(output[j])
                all_values.append(temp)
            best_output = dummy(all_values)
            filename = 'HWA_'+str(best_output[0])+'_'+str(best_output[2])+'_'+str(best_output[3])+'_'+str(best_output[4])+'_'+str(best_output[5])+'.csv'
            best_fit[best_output[5]] = filename
            write_to_folder(best_output[1], opDirectory, filename, actual_dir)
            for i in range(int(len(y)//2)):
                output = multiplicative(y, int(i+1), fc)
                temp = [i+1]
                for j in range(0,len(output)):
                    temp.append(output[j])
                all_values.append(temp)
            best_output = dummy(all_values)
            filename = 'HWM_'+str(best_output[0])+'_'+str(best_output[2])+'_'+str(best_output[3])+'_'+str(best_output[4])+'_'+str(best_output[5])+'.csv'
            best_fit[best_output[5]] = filename
            write_to_folder(best_output[1], opDirectory, filename, actual_dir)
            for i in range(int(len(y)//2)):
                output = linear(y, int(i+1), fc)
                for j in range(0,len(output)):
                    temp.append(output[j])
                all_values.append(temp)            
            best_output = dummy(all_values)
            filename = 'HWL_'+str(best_output[0])+'_'+str(best_output[2])+'_'+str(best_output[3])+'_'+str(best_output[4])+'.csv'
            best_fit[best_output[4]] = filename
            write_to_folder(best_output[1], opDirectory, filename, actual_dir)
            return best_fit[min(best_fit.keys)]
        else:
            if(method == 'HWA'):
                if(c == None):
                    if(arguments == None):
                        for i in range(int(len(y)//2)):
                            output = additive(y, int(i+1), fc)
                            temp = [i+1]
                            for j in range(0,len(output)):
                                temp.append(output[j])
                            all_values.append(temp)
                        best_output = dummy(all_values)
                        filename = 'HWA_'+str(best_output[0])+'_'+str(best_output[2])+'_'+str(best_output[3])+'_'+str(best_output[4])+'_'+str(best_output[5])+'.csv'
                        write_to_folder(best_output[1], opDirectory, filename, actual_dir)
                        return filename
                    else:
                        alpha, beta, gamma = arguments
                        for i in range(int(len(y)//2)):
                            output = additive(y, int(i+1), fc, alpha, beta, gamma)
                            temp = [i+1]
                            for j in range(0,len(output)):
                                temp.append(output[j])
                            all_values.append(temp)
                        best_output = dummy(all_values)
                        filename = 'HWA_'+str(best_output[0])+'_'+str(best_output[2])+'_'+str(best_output[3])+'_'+str(best_output[4])+'_'+str(best_output[5])+'.csv'
                        write_to_folder(best_output[1], opDirectory, filename, actual_dir)
                        return filename
                else:
                    if (arguments == None):
                        best_output = additive(y,c,fc)
                        filename = 'HWA_'+str(c)+'_'+str(best_output[1])+'_'+str(best_output[2])+'_'+str(best_output[3])+'_'+str(best_output[4])+'.csv'
                        write_to_folder(best_output[0], opDirectory, filename, actual_dir)
                        return filename
                    else:
                        alpha, beta, gamma = arguments
                        best_output = additive(y,c,fc, alpha, beta, gamma)
                        filename = 'HWA_'+str(c)+'_'+str(best_output[1])+'_'+str(best_output[2])+'_'+str(best_output[3])+'_'+str(best_output[4])+'.csv'
                        write_to_folder(best_output[0], opDirectory, filename, actual_dir)
                        return filename
            elif(method == 'HWM'):
                if(c == None):
                    if(arguments == None):
                        for i in range(int(len(y)//2)):
                            output = multiplicative(y, int(i+1), fc)
                            temp = [i+1]
                            for j in range(0,len(output)):
                                temp.append(output[j])
                            all_values.append(temp)
                        best_output = dummy(all_values)
                        filename = 'HWM_'+str(best_output[0])+'_'+str(best_output[2])+'_'+str(best_output[3])+'_'+str(best_output[4])+'_'+str(best_output[5])+'.csv'
                        write_to_folder(best_output[1], opDirectory, filename, actual_dir)
                        return filename
                    else:
                        alpha, beta, gamma = arguments
                        for i in range(int(len(y)//2)):
                            output = multiplicative(y, int(i+1), fc, alpha, beta, gamma)
                            temp = [i+1]
                            for j in range(0,len(output)):
                                temp.append(output[j])
                            all_values.append(temp)
                        best_output = dummy(all_values)
                        filename = 'HWM_'+str(best_output[0])+'_'+str(best_output[2])+'_'+str(best_output[3])+'_'+str(best_output[4])+'_'+str(best_output[5])+'.csv'
                        write_to_folder(best_output[1], opDirectory, filename, actual_dir)
                        return filename
                else:
                    if (arguments == None):
                        best_output = multiplicative(y,c,fc)
                        filename = 'HWM_'+str(c)+'_'+str(best_output[1])+'_'+str(best_output[2])+'_'+str(best_output[3])+'_'+str(best_output[4])+'.csv'
                        write_to_folder(best_output[0], opDirectory, filename, actual_dir)
                        return filename
                    else:
                        alpha, beta, gamma = arguments
                        best_output = multiplicative(y,c,fc, alpha, beta, gamma)
                        filename = 'HWM_'+str(c)+'_'+str(best_output[1])+'_'+str(best_output[2])+'_'+str(best_output[3])+'_'+str(best_output[4])+'.csv'
                        write_to_folder(best_output[0], opDirectory, filename, actual_dir)
                        return filename
                        
            elif(method == 'HWL'):
                if(arguments == None):
                    best_output = linear(y, fc)
                    filename = 'HWL_'+str(best_output[1])+'_'+str(best_output[2])+'_'+str(best_output[3])+'.csv'
                    write_to_folder(best_output[0], opDirectory, filename, actual_dir)
                    return filename
                else:
                    alpha, beta = arguments
                    best_output = linear(y, fc, alpha, beta)
                    filename = 'HWL_'+str(best_output[1])+'_'+str(best_output[2])+'_'+str(best_output[3])+'.csv'
                    write_to_folder(best_output[0], opDirectory, filename, actual_dir)
                    return filename

In [ ]:
method_dict = {}
method_dict['filename'] = 'input_demand.csv'
prana_forecast(method_dict)

['input_demand', 'csv']
YES
shariqN
